## Fine-tune Inception V3 / DenseNet121

- Define generator to recover dataset and extract label. Source: https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly

In [1]:
from skimage.io import imread
from skimage.transform import resize
import numpy as np
from keras.utils import Sequence
from skimage.transform import resize

class TAS_Generator(Sequence):

    def __init__(self, data_path='INSA_data_images/', prefix='test', batch_size=10):
        
        # load dataset
        self.x = np.load(data_path + prefix + '_RGB_0_10_25.npy')
        self.y = np.load(data_path + prefix + '_labels_0_10_25.npy')
        
        # reshape image
        for i in range(len(self.x)):
            self.x[i] = resize(self.x[i], (224, 224))
        
        self.batch_size = batch_size

    def __len__(self):
        return int ( np.ceil(self.x.shape[0] / float(self.batch_size)) )

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

        return np.array([batch_x, batch_y])

/usr/local/insa/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


- Fine-turn generator. Source: https://keras.io/applications/#fine-tune-inceptionv3-on-a-new-set-of-classes

- To avoid reload multiple times, the base model will be called once:

In [8]:
import keras; print(keras.__version__)

2.2.0


In [6]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.densenet import DenseNet121

from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

# create the base pre-trained model
base_model = DenseNet121(weights='imagenet',
                         input_shape = (32,32,3),
                         include_top=False)

ValueError: Input size must be at least 221x221; got `input_shape=(32, 32, 3)`

In [5]:
from keras.optimizers import SGD

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(200, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# custom generator to load our own data
train_gen = TAS_Generator('INSA_data_images/', 'test', 20)
test_gen = TAS_Generator('INSA_data_images/', 'test', 20)

# train the model on the new data for a few epochs
model.fit_generator(generator = train_gen,
                    verbose = 1,
                    use_multiprocessing=True,
                    workers=6)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(generator = test_gen,
                    verbose = 1,
                    use_multiprocessing=True, 
                    workers=6)

/usr/local/insa/anaconda/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


ValueError: could not broadcast input array from shape (224,224,3) into shape (32,32,3)